In [1]:
import cv2 
import numpy as np
from collections import deque


In [2]:
def setValues(x): 
   print("") 
   
  
# Creating the trackbars needed for  
# adjusting the marker colour These  
# trackbars will be used for setting  
# the upper and lower ranges of the 
# HSV required for particular colour 
cv2.namedWindow("Color detectors") 
cv2.createTrackbar("Upper Hue", "Color detectors", 
                   153, 180, setValues) 
cv2.createTrackbar("Upper Saturation", "Color detectors", 
                   255, 255, setValues) 
cv2.createTrackbar("Upper Value", "Color detectors",  
                   255, 255, setValues) 
cv2.createTrackbar("Lower Hue", "Color detectors", 
                   64, 180, setValues) 
cv2.createTrackbar("Lower Saturation", "Color detectors",  
                   72, 255, setValues) 
cv2.createTrackbar("Lower Value", "Color detectors",  
                   49, 255, setValues) 

In [3]:
#color pallete in BGR
                
color_lst = [(255,0,0) , (0,255,0), (0,0,255), (0,0,0) ,(255,255,255)]

idx = 0

lst=[]

# green_id = 0
# blue_id = 0
# black_id = 0
# red_id =0
#paint window
paintWindow = np.zeros((471, 636, 3)) + 255

cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE) 

kernel = np.ones((5, 5), np.uint8) 

In [4]:
cap =cv2.VideoCapture(0)

while True:
    
    ret,frame =cap.read()
    frame = cv2.flip(frame,1) #camera sees in opposite way
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    u_h = cv2.getTrackbarPos("Upper Hue", 
                               "Color detectors") 
    u_s = cv2.getTrackbarPos("Upper Saturation", 
                                      "Color detectors") 
    u_v = cv2.getTrackbarPos("Upper Value", 
                                 "Color detectors") 
    l_h = cv2.getTrackbarPos("Lower Hue", 
                               "Color detectors") 
    l_s = cv2.getTrackbarPos("Lower Saturation", 
                                      "Color detectors") 
    l_v = cv2.getTrackbarPos("Lower Value", 
                                 "Color detectors") 

    Upper_hsv = np.array([u_h,u_s,u_v]) 
    Lower_hsv = np.array([l_h,l_s,l_v]) 
    
    mask = cv2.inRange(hsv,Lower_hsv,Upper_hsv)
    mask = cv2.erode(mask, kernel, iterations = 1) 
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel) 
    mask = cv2.dilate(mask, kernel, iterations = 1) 
    
    cnts,_ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
    center = None
    
    frame = cv2.rectangle(frame, (40, 1), (140, 65),  
                          (122, 122, 122), -1) 
    frame = cv2.rectangle(frame, (160, 1), (255, 65), 
                          color_lst[0], -1) 
    frame = cv2.rectangle(frame, (275, 1), (370, 65),  
                          color_lst[1], -1) 
    frame = cv2.rectangle(frame, (390, 1), (485, 65),  
                          color_lst[2], -1) 
    frame = cv2.rectangle(frame, (505, 1), (600, 65), 
                          color_lst[3], -1) 
      
    cv2.putText(frame, "CLEAR ALL", (49, 33), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                (255, 255, 255), 2, cv2.LINE_AA) 
    cv2.putText(frame, "BLUE", (185, 33),  
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                (255, 255, 255), 2, cv2.LINE_AA) 
      
    cv2.putText(frame, "GREEN", (298, 33), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                (255, 255, 255), 2, cv2.LINE_AA) 
      
    cv2.putText(frame, "RED", (398, 33), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                (255, 255, 255), 2, cv2.LINE_AA) 
    cv2.putText(frame, "BLACK", (520, 33), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5,  
                (255,255,255),2,cv2.LINE_AA) 
    
      
    
    if(len(cnts)) > 0:
        
        cnt = sorted(cnts,key = cv2.contourArea , reverse = True)[0]
        ((x,y),radius) = cv2.minEnclosingCircle(cnt) 
        
        cv2.circle(frame , (int (x),int (y)),int(radius),(0,255,255),2)
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
        
        if center[1]<=65:
            
            if 40<=center[0]<=140:
                lst=[]
                paintWindow[67:, :, :] = 255 
            elif 160 <= center[0] <= 255: 
                    idx = 0 # Blue 
            elif 275 <= center[0] <= 370: 
                    idx = 1 # Green 
            elif 390 <= center[0] <= 485: 
                    idx = 2 # Red 
            elif 505 <= center[0] <= 600: 
                    idx = 3 # Balck
        else:
            lst.append(center)
            
    else:
        lst=[] 
    
    if len(lst)!=0: 
            cv2.line(frame, lst[len(lst)-2], lst[len(lst)-1], color_lst[idx], 2) 
            cv2.line(paintWindow,lst[len(lst)-2] , lst[len(lst)-1], color_lst[idx], 2) 

            
    
    cv2.imshow("Tracking", frame) 
    cv2.imshow("Paint", paintWindow) 
    cv2.imshow("mask", mask) 
   
    # If the 'q' key is pressed then stop the application  
    if cv2.waitKey(1) & 0xFF == ord("q"): 
        break
  
# Release the camera and all resources 
cap.release() 
cv2.destroyAllWindows() 
